## College Football Data API Stuff

In [1]:
import bs4
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from time import sleep
import cfbd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = 'cCUuy68SmBD3qQ/HK1HZ0cks/gHrSkPiSUdcom44hDyV49P3syOpoiFJnGq8MDDY'
configuration.api_key_prefix['Authorization'] = 'Bearer'

api_config = cfbd.ApiClient(configuration)

In [3]:
api_instance = cfbd.GamesApi(cfbd.ApiClient(configuration))
#api_response = api_instance.get_team_game_stats(year=2023, week=1, classification="fbs")
#print(api_response[0])

In [11]:
games_2023 = api_instance.get_team_game_stats(year=2023, week = 1, classification="fbs")

In [12]:
print(len(games_2023))

94


In [17]:
game = games_2023[0]

In [18]:
print(games_2023[0].teams[0].stats)

[{'category': 'rushingTDs', 'stat': '0'}, {'category': 'passingTDs', 'stat': '0'}, {'category': 'kickReturnYards', 'stat': '12'}, {'category': 'kickReturnTDs', 'stat': '0'}, {'category': 'kickReturns', 'stat': '1'}, {'category': 'kickingPoints', 'stat': '3'}, {'category': 'fumblesRecovered', 'stat': '0'}, {'category': 'totalFumbles', 'stat': '3'}, {'category': 'tacklesForLoss', 'stat': '2'}, {'category': 'defensiveTDs', 'stat': '0'}, {'category': 'tackles', 'stat': '42'}, {'category': 'sacks', 'stat': '0'}, {'category': 'qbHurries', 'stat': '1'}, {'category': 'passesDeflected', 'stat': '2'}, {'category': 'possessionTime', 'stat': '33:14'}, {'category': 'interceptions', 'stat': '0'}, {'category': 'fumblesLost', 'stat': '0'}, {'category': 'turnovers', 'stat': '0'}, {'category': 'totalPenaltiesYards', 'stat': '3-26'}, {'category': 'yardsPerRushAttempt', 'stat': '2.6'}, {'category': 'rushingAttempts', 'stat': '48'}, {'category': 'rushingYards', 'stat': '126'}, {'category': 'yardsPerPass', 

In [14]:
#print(games_2023[0].teams[0])
stats = games_2023[0].teams[0].stats
print(stats)
newdict = {}
newdict[stats[0].category] = stats[0].stat
for stat in stats:
    newdict[stat.category] = stat.stat
print(newdict)

[{'category': 'rushingTDs', 'stat': '0'}, {'category': 'passingTDs', 'stat': '0'}, {'category': 'kickReturnYards', 'stat': '12'}, {'category': 'kickReturnTDs', 'stat': '0'}, {'category': 'kickReturns', 'stat': '1'}, {'category': 'kickingPoints', 'stat': '3'}, {'category': 'fumblesRecovered', 'stat': '0'}, {'category': 'totalFumbles', 'stat': '3'}, {'category': 'tacklesForLoss', 'stat': '2'}, {'category': 'defensiveTDs', 'stat': '0'}, {'category': 'tackles', 'stat': '42'}, {'category': 'sacks', 'stat': '0'}, {'category': 'qbHurries', 'stat': '1'}, {'category': 'passesDeflected', 'stat': '2'}, {'category': 'possessionTime', 'stat': '33:14'}, {'category': 'interceptions', 'stat': '0'}, {'category': 'fumblesLost', 'stat': '0'}, {'category': 'turnovers', 'stat': '0'}, {'category': 'totalPenaltiesYards', 'stat': '3-26'}, {'category': 'yardsPerRushAttempt', 'stat': '2.6'}, {'category': 'rushingAttempts', 'stat': '48'}, {'category': 'rushingYards', 'stat': '126'}, {'category': 'yardsPerPass', 

In [19]:
def process_stats(stats):
    stat_dict = {}
    print(stats)
    for stat in stats:
        stat_dict[stat.category] = stat.stat
    return stat_dict

In [25]:
def get_home_dict(home_data):
    home_dict = {}
    home_dict['home_school'] = home_data.school
    home_dict['home_school_id'] = home_data.school_id
    home_dict['home_points'] = home_data.points
    home_stats = home_data.stats
    stat_dict = process_stats(home_stats)
    stat_dict = {f"home_{key}": val for key, val in stat_dict.items()}

    home_dict.update(stat_dict)
    return home_dict


In [12]:
def game_processing(game):
    game_dict = {'id':game.id}
    if game.teams[0].home_away == "away":
        home_dict = get_home_dict(game.teams[0])
        away_dict = get_away_dict(game.teams[1])
    else:
        home_dict = get_home_dict(game.teams[1])
        away_dict = get_away_dict(game.teams[0])
    game_dict = home_dict.update(away_dict)
    game_dict['id'] = game.id

    return game_dict
    
    

In [26]:
test = get_home_dict(game.teams[0])
print(test)

[{'category': 'rushingTDs', 'stat': '0'}, {'category': 'passingTDs', 'stat': '0'}, {'category': 'kickReturnYards', 'stat': '12'}, {'category': 'kickReturnTDs', 'stat': '0'}, {'category': 'kickReturns', 'stat': '1'}, {'category': 'kickingPoints', 'stat': '3'}, {'category': 'fumblesRecovered', 'stat': '0'}, {'category': 'totalFumbles', 'stat': '3'}, {'category': 'tacklesForLoss', 'stat': '2'}, {'category': 'defensiveTDs', 'stat': '0'}, {'category': 'tackles', 'stat': '42'}, {'category': 'sacks', 'stat': '0'}, {'category': 'qbHurries', 'stat': '1'}, {'category': 'passesDeflected', 'stat': '2'}, {'category': 'possessionTime', 'stat': '33:14'}, {'category': 'interceptions', 'stat': '0'}, {'category': 'fumblesLost', 'stat': '0'}, {'category': 'turnovers', 'stat': '0'}, {'category': 'totalPenaltiesYards', 'stat': '3-26'}, {'category': 'yardsPerRushAttempt', 'stat': '2.6'}, {'category': 'rushingAttempts', 'stat': '48'}, {'category': 'rushingYards', 'stat': '126'}, {'category': 'yardsPerPass', 

In [24]:
print(test)

None
